<a href="https://colab.research.google.com/github/lyj7518/dacon/blob/main/%EC%9C%A0%EC%A7%84_feature_engineering_catboost_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #1. [person_prefer_d_2_attribute_d_m] 제거

In [2]:
from google import colab
colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install catboost

In [4]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 
import optuna

from catboost import Pool,CatBoostClassifier

In [5]:
train_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/test.csv')
SEED = 14
code_d = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_H_코드.csv')
code_l = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [6]:
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m"]
code_h.columns= ["attribute_h","attribute_h_m","attribute_h_l"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [7]:
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:  #매개변수의 타입이 data.frame
#print(test.__annotations__) 참고로 import annotation해서 함수안 파라미터를 확인할 수 있다.
    df = df.copy()
    # df는 데이터 파일이 들어올 변수로 설정
    df_code = df_code.copy()
    # 다른 속성코드 데이터 로드
    df_code = df_code.add_prefix(f"{col}_")
    # 로드 데이터 마다 col명을 달고 인덱스를 시작하고자 ex) col_1  col_2 이런식
    df_code.columns.values[0] = col # col = attribute_l 이런 식으로 큰 이름만 따온다.
    return pd.merge(df,df_code,how="left",on=col) 
    # col기준으로 left조인, left는 df// 따라서 df기준으로 조인

def preprocess_data(
                    df:pd.DataFrame,
                    is_train:bool = True,
                    cols_merge:List[Tuple[str,pd.DataFrame]] = []  ,
                    cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "person_prefer_d_2_attribute_d_m"]
                    )->Tuple[pd.DataFrame,np.ndarray]: #Tuple형태로 리턴하겠다.
    df = df.copy()

    y_data = None
    if is_train: # 처음에 value는 true이니 진행
        y_data = df["target"].to_numpy()
        #numpy값으로 target값을 y_data에 저장
        df = df.drop(columns="target")
        #target컬럼 제거

    for col, df_code in cols_merge:
        df = merge_codes(df,df_code,col)
        #left 조인함

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

    df = df.drop(columns=cols_drop)
    return (df , y_data)
    #하나로 다 합쳐서 조인함

In [8]:
# 이 부분 유진이랑 수정하기

# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_d" , "contents_attribute_d_attribute_d_d"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_d" , "contents_attribute_d_attribute_d_d"),

    ("person_prefer_h_1_attribute_h_m" , "contents_attribute_h_attribute_h_m"),#attribute_h_m ,l,none
    ("person_prefer_h_1_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_2_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_3_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_2_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_3_attribute_h_l" , "contents_attribute_h_attribute_h_l"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "person_prefer_d_2_attribute_d_m" ]

In [9]:
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_d" , "contents_attribute_d_attribute_d_d"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_d" , "contents_attribute_d_attribute_d_d"),

    ("person_prefer_h_1_attribute_h_m" , "contents_attribute_h_attribute_h_m"),#attribute_h_m ,l,none
    ("person_prefer_h_1_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_2_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_3_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_2_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_3_attribute_h_l" , "contents_attribute_h_attribute_h_l"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g", "contents_open_dt", "person_prefer_d_2_attribute_d_m" ]

In [10]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
#데이터 조인해서 train_data를 x와 y로 분리
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
#test에서는 타겟 값이 없으니 _로 표기
x_train.shape , y_train.shape , x_test.shape

((501951, 67), (501951,), (46404, 67))

In [11]:
is_holdout = False
#기존 train과 val데이터 분할 안함
n_splits = 5
#반복수
patience = 50
#최소 인내수
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
#k-fold 5번 분할하고 교차 검증 진행하기 위함

In [12]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()
#날짜나 선호 특성에 대한 값이 낮은것(범주가 최대 2개인것)들을 제외하기 위함 

### Optuna

In [14]:
from sklearn.model_selection import train_test_split

In [13]:
! pip install optuna

In [14]:
# create trial function
OPTUNA_OPTIMIZATION = True

def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(x_train,y_train, test_size=0.3)
    
    params = {
        'iterations':trial.suggest_int("iterations", 500, 3000),
        'objective':trial.suggest_categorical('objcetive',['CrossEntropy','Logloss']),
        'bootstrap_type':trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'od_wait':trial.suggest_int('od_wait', 500, 1000),
        #The number of iterations to continue the training early_stopping_rounds같은거 
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01,1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',20,50),
        'depth': trial.suggest_int('depth',1,15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,20),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose': False,
        "eval_metric":'F1',
        "one_hot_max_size":trial.suggest_int("one_hot_max_size",1,5),
        'task_type' : 'GPU',
    }
    
    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)
    
    model = CatBoostClassifier(**params)
    model.fit(
        train_x, train_y, eval_set=[(valid_x, valid_y)],
        #early_stopping_rounds=200,
        use_best_model=True
    )
    
    # validation prediction

    preds = model.predict(valid_x)
    pred_labels = np.rint(preds)
    score = f1_score(valid_y, pred_labels)
    return score

In [20]:
study = optuna.create_study(
    direction='maximize',
    study_name='CatbClf'
)

study.optimize(
    objective, 
    n_trials=100
)

[I 2022-01-05 01:36:27,298] A new study created in memory with name: CatbClf
[I 2022-01-05 01:36:57,106] Trial 0 finished with value: 0.6159132757592262 and parameters: {'iterations': 1605, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bayesian', 'od_wait': 875, 'learning_rate': 0.6506696613378945, 'reg_lambda': 75.59032475007588, 'random_strength': 33.20882980414596, 'depth': 9, 'min_data_in_leaf': 2, 'leaf_estimation_iterations': 1, 'one_hot_max_size': 1, 'bagging_temperature': 9.411796153216915}. Best is trial 0 with value: 0.6159132757592262.
[I 2022-01-05 01:37:47,813] Trial 1 finished with value: 0.6274550168224541 and parameters: {'iterations': 1938, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bernoulli', 'od_wait': 761, 'learning_rate': 0.49241792509974147, 'reg_lambda': 59.86125677654717, 'random_strength': 46.273045591259454, 'depth': 11, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 9, 'one_hot_max_size': 5, 'subsample': 0.8062891102245889}. Best is trial 1 with

In [21]:
Best_params = study.best_trial.params
dic2 = {"task_type":'GPU',"eval_metric":'F1',"cat_features":cat_features,"one_hot_max_size":4}
Best_params.update(dic2)
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

Best Trial: 0.6468582500015934
Best Params: {'iterations': 1100, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bernoulli', 'od_wait': 739, 'learning_rate': 0.0712062295412846, 'reg_lambda': 68.24093295080826, 'random_strength': 39.1425811871665, 'depth': 10, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 14, 'one_hot_max_size': 1, 'subsample': 0.9761466040433332}


### catboost 진행

In [15]:
best_params= {'iterations': 1100, 'objective': 'CrossEntropy', 
              'bootstrap_type': 'Bernoulli', 'od_wait': 739, 'learning_rate': 0.0712062295412846, 
              'reg_lambda': 68.24093295080826, 'random_strength': 39.1425811871665,
              'depth': 10, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 14, 'one_hot_max_size': 1,
              'subsample': 0.9761466040433332,
               "cat_features": cat_features,
               "eval_metric":'F1',
               'task_type' : 'GPU'}

In [ ]:
scores = []
models = []


for tri, vai in cv.split(x_train):
    preds = []
    model = CatBoostClassifier(**best_params)
    model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds = 500,
            verbose=100
        )
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break    

0:	learn: 0.6058201	test: 0.6113634	best: 0.6113634 (0)	total: 658ms	remaining: 12m 2s
100:	learn: 0.6669303	test: 0.6730766	best: 0.6731209 (99)	total: 49.7s	remaining: 8m 11s
200:	learn: 0.6794327	test: 0.6745398	best: 0.6760002 (178)	total: 1m 43s	remaining: 7m 45s
300:	learn: 0.6910901	test: 0.6713445	best: 0.6760002 (178)	total: 2m 32s	remaining: 6m 44s
400:	learn: 0.7002310	test: 0.6729199	best: 0.6760002 (178)	total: 3m 18s	remaining: 5m 46s
500:	learn: 0.7081771	test: 0.6747501	best: 0.6760002 (178)	total: 4m	remaining: 4m 47s
600:	learn: 0.7156074	test: 0.6743849	best: 0.6760002 (178)	total: 4m 43s	remaining: 3m 55s
bestTest = 0.6760001557
bestIteration = 178
Shrink model to first 179 iterations.
0:	learn: 0.6021406	test: 0.6043634	best: 0.6043634 (0)	total: 557ms	remaining: 10m 12s
100:	learn: 0.6678608	test: 0.6673210	best: 0.6678660 (83)	total: 50.4s	remaining: 8m 18s
200:	learn: 0.6793286	test: 0.6703897	best: 0.6720598 (184)	total: 1m 38s	remaining: 7m 19s
300:	learn: 0.6

In [ ]:
'''scores = []
models = []


for tri, vai in cv.split(x_train):
    preds = []
    model = CatBoostClassifier(iterations=2401, objective= 'CrossEntropy', 
              bootstrap_type= 'MVS',od_wait= 623, learning_rate= 0.3250307462792831, 
              reg_lambda= 73.41218413701267, random_strength= 22.46080042848193,
              depth= 1, min_data_in_leaf= 16, leaf_estimation_iterations= 15, one_hot_max_size= 5)
    model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds = 500,
            verbose=100
        )
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break    '''

        

'scores = []\nmodels = []\n\n\nfor tri, vai in cv.split(x_train):\n    preds = []\n    model = CatBoostClassifier(iterations=2401, objective= \'CrossEntropy\', \n              bootstrap_type= \'MVS\',od_wait= 623, learning_rate= 0.3250307462792831, \n              reg_lambda= 73.41218413701267, random_strength= 22.46080042848193,\n              depth= 1, min_data_in_leaf= 16, leaf_estimation_iterations= 15, one_hot_max_size= 5)\n    model.fit(x_train.iloc[tri], y_train[tri],\n            eval_set=[(x_train.iloc[vai], y_train[vai])],\n            early_stopping_rounds = 500,\n            verbose=100\n        )\n    models.append(model)\n    scores.append(model.get_best_score()["validation"]["F1"])\n    if is_holdout:\n        break    '

In [27]:
print(scores)
print(np.mean(scores))

[0.6331433355875439, 0.6099732268715173, 0.6366636567766475, 0.6524674267692733, 0.6282777168335753]
0.6321050725677114


In [28]:
threshold = 0.385
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

[0.667476340275289, 0.6657983746104315, 0.6656653729597534, 0.6652839545566347, 0.6739349257304532]
0.6676317936265124


In [ ]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)